In [20]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as c
import sympy as sy
import scipy.constants as pc
from sympy import *
from math import log10, floor

In [2]:
# Funktionen
def am(werte):
    return np.mean(werte)

def std(werte):
    summen_der_quadrate = 0
    for wert in werte:
        summen_der_quadrate+=(wert-am(werte))**2       
    return np.sqrt(summen_der_quadrate/(werte.size-1))

def stu(werte):
    return std(werte)/np.sqrt(werte.size)

def fehler(f, werte, unsicherheiten, variablen):
    zMax = 0
    for i in range(werte.size):
        dif = np.abs(sy.diff(f, variablen[i]))
        zMax += (dif * unsicherheiten[i])**2
    zMax = sqrt(zMax)
    for j in range(werte.size):
        zMax = sub(zMax, variablen[j], werte[j])
    return zMax

def fehlerMax(f, werte, unsicherheiten, variablen):
    zMax = 0
    for i in range(werte.size):
        dif = np.abs(sy.diff(f, variablen[i]))
        zMax += dif * unsicherheiten[i]
    for j in range(werte.size):
        zMax = sub(zMax, variablen[j], werte[j])
    return zMax

def groessterBeitrag(f, werte, unsicherheiten, variablen):
    zMax = 0
    zMaxArray = np.array([])
    for i in range (werte.size):
        zMax = (np.abs(sy.diff(f, variablen[i])) * unsicherheiten[i])
        for j in range(werte.size):
            zMax = sub(zMax, variablen[j], werte[j])
        x = np.array([zMax])
        zMaxArray = np.append(zMaxArray, x)
    return variablen[np.argmax(zMaxArray)]

def sub(zMax, variable, wert):
    return zMax.subs([(variable, wert)])

def einfachGaussAdd(unsicherheiten, koeffizienten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (unsicherheiten[i]*unsicherheiten[i])**2
    return sqrt(dz)

def einfachGaussMult(unsicherheiten, werte, exponenten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (exponenten[i]*unsicherheiten[i]/werte[i])**2
    return sqrt(dz)

def einfachGaussMultMax(unsicherheiten, werte, exponenten):
    max_a = 0
    num = -1
    for i in range (unsicherheiten.size):
        maxi = (exponenten[i]*unsicherheiten[i]/werte[i])**2
        if max_a <= maxi:
            max_a = maxi
            num = i
    return (max_a, num)

def vertraeglichkeit(am_1, am_2, stu_1, stu_2):
    return (np.abs(am_1-am_2)/(sqrt((stu_1**2)*(stu_2**2))))

def bestA(x, y):
    return (sum(x**2)*sum(y)-sum(x)*sum(x*y))/(x.size*sum(x**2)-sum(x)**2)

def bestB(x, y):
    return (x.size*sum(x*y)-sum(x)*sum(y))/(x.size*sum(x**2)-sum(x)**2)

def streuung(x, y):
    s = 0
    for i in range(x.size):
        s += (y[i]+(bestA(x, y)+bestB(x, y)*x[i]))**2
    return np.sqrt(s/(x.size-2))

def stuA(x, y):
    s = streuung(x, y)
    return s*np.sqrt(sum(x**2)/((x.size*sum(x**2)-sum(x)**2)))

def stuB(x, y):
    s = streuung(x, y)
    return s*np.sqrt(x.size/((x.size*sum(x**2)-sum(x)**2)))



In [3]:
# s(m)
m = np.array([50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 
              550, 600, 650, 700, 750, 800, 850, 900]) #0.1g
mess = np.array([0.44, 0.81, 1.22, 1.61, 2.02, 2.43, 2.79, 3.13, 3.60, 3.97, 
                 4.39, 4.74, 5.15, 5.52, 5.87, 6.36, 6.76, 7.11]) # 0.1
alu = np.array([0.19, 0.40, 0.57, 0.80, 0.96, 1.18, 1.39, 1.59, 1.77, 2.0, 
                2.16, 2.45, 2.58, 2.77, 2.97, 3.15, 3.38, 3.56]) # 0.1
stahl = np.array([0.07, 0.13, 0.19, 0.27, 0.32, 0.39, 0.44, 0.50, 0.57, 0.63, 
                  0.69, 0.75, 0.81, 0.89, 0.95, 1.02, 1.08, 1.14]) #0.1

# Verschiedene d
m_short = 450 # 0.1g
d = np.array([75.1, 71.3, 67.1, 63.2, 59.3, 54.7, 50.3, 46.7, 43.3, 37.1, 33.3]) # 0.1
s = np.array([0.89, 0.76, 0.65, 0.54, 0.45, 0.34, 0.29, 0.22, 0.20, 0.12, 0.07]) #0.1

# Fläche-Biege
a_r = np.array([10.25, 10.15, 10.15, 10.10, 12.20, 10.15, 10.10, 12.10, 10.95, 10.15])
i_r = np.array([8.05, 7.0, 5.85, 4.0, 10.0, 0, 0, 0, 6.9, 6.8])
s_f = np.array([1.24, 0.94, 0.78, 0.77, 0.65, 0.69, 0.79, 0.34, 0.97, 0.76])
flaeche = pc.pi*(a_r**2-i_r**2)
biege = pc.pi*(a_r**4-i_r**4)

In [4]:
#Fehler 

material = np.array(["Messing", "Aluminium", "Stahl"])

m_über_s = np.array([127.22,251.3,794])
m_über_s_f = np.array([0.42, 1.3, 4])

g = 9.808
g_f = 0.001

d = 84.8
d_f = 2

a = 6
a_f = 0.1

b = 10
b_f = 0.1

#Elastizitätsmopdule

elamodule = np.zeros(3)
for i in range(3):
    elamodule[i] = 0.25 * (m_über_s[i] * g * d**3) / (a * b**3)
    print("Für", material[i], "ist das Elaszitätsmodul", elamodule[i],";")

#Fehlerberechnung:
# m_über_s, g, d, a, b, 
for i in range(3):
    expo_ela = np.array([1,1,3,-1,-3])
    werte_ela = np.array([m_über_s[i], g, d, a, b])
    unsich_ela = np.array([m_über_s_f[i], g_f, d_f, a_f, b_f])
    print("Für",material[i],"ist der Fehler", elamodule[i] * einfachGaussMult(unsich_ela, werte_ela, expo_ela),";")
    

   


Für Messing ist das Elaszitätsmodul 31703.86160085674 ;
Für Aluminium ist das Elaszitätsmodul 62625.219464669855 ;
Für Stahl ist das Elaszitätsmodul 197868.77936708264 ;
Für Messing ist der Fehler 2495.34107648517 ;
Für Aluminium ist der Fehler 4935.39938501654 ;
Für Stahl ist der Fehler 15592.0063411817 ;


In [8]:
for i in range(3):
    max_a, num = einfachGaussMultMax(unsich_ela, werte_ela, expo_ela)
    print("Für",material[i], max_a*elamodule[i] , num)

Für Messing 158.71682227199994 2
Für Aluminium 313.5162508799999 2
Für Stahl 990.5766143999997 2
